In [ ]:
# Run model

pip install vllm

CUDA_VISIBLE_DEVICES=1,2,3 python -m vllm.entrypoints.openai.api_server --model google/gemma-1.1-7b-it --dtype auto --api-key token-abc123 --max-model-len 2048


/red-phantasm-efs/rewrites/llama2-gec-sota/llama-2-7b-chat-hf-batch-4-acc-2-lr-1e-05-up-800-wup-100-seed-9453-gec_sota-gec_sota-bea-all/models
/red-phantasm-efs/rewrites/llama2-gec-sota/llama-2-13b-hf-batch-4-acc-2-lr-1e-05-up-800-wup-100-seed-9453-gec_sota-gec_sota-bea-all/models


In [1]:
# Generate samples

In [ ]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [ ]:
df = pd.read_csv('troy-blogs.train.tokenized.csv')
df_sample = df.sample(50000)

In [ ]:
from openai import OpenAI
client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="token-abc123",
)


In [ ]:
def infer(text):
    completion = client.chat.completions.create(
      # model="mistralai/Mistral-7B-Instruct-v0.2",
        model="gec-llama2-7b-public/",
        temperature=1,
        n=5,
        messages=[
            {"role": "system", "content": "Rewrite this text to make it grammatically correct ."},
            {"role": "user", "content": text}
        ]
    )
    preds = []
    for choise in completion.choices:
        pred = choise.message.content
        prefixes = ["text:", "text is:", ":\n\n", ":\n"]
        for p in prefixes:
            if p in pred:
                pred = pred[pred.index(p)+len(p)+1:]
                pred = pred.lstrip()
                if "\n" in pred:
                    pred = pred[:pred.index('\n')]
                break
        preds.append(pred)
    return preds

infer("I hop ths mesage find u ..")

In [ ]:
df_sample['pred-llama2-sample'] = df_sample.src.progress_map(infer)

In [ ]:
df_sample.to_json('troy-llama7-gec-20k-sample.json')

In [ ]:
# Score samples with Greco / Scibelty

In [ ]:
import os
import numpy as np
import argparse
from tqdm import tqdm
from collections import Counter

import torch
from greco_model import GRECO


def get_top_candidate(source, hyps, sys_names, model, voting_bias=True, m7_bias=0, verbose=0):
    if len(hyps) == 1:
        return hyps[0], sys_names[0]

    
    # score hypothesis
    scores = model.score(source, hyps)
    scores = scores.tolist()
    
    # get sys_weights based on majority voting
    # sys_proportions = [len(x.split("|")) for x in sys_names]
    # sys_proportions = [len(x.split("|")) + (1 if "orig" in x else 0) for x in sys_names]
    sys_proportions = [len(x.split("|")) + (m7_bias if "ens_m" in x else 0) for x in sys_names]
    # TODO: find out weighting factor
    wf = sum(sys_proportions)
    wn = wf/len(sys_proportions)
    mn = max(sys_proportions)
    # voting_bias = 1
    # sys_proportions = [(x )/wf for x in sys_proportions]
    # sys_proportions = [np.log(x + 1) for x in sys_proportions]
    # sys_proportions = [(x)/wn for x in sys_proportions]
    sys_proportions = [(x)/mn for x in sys_proportions]
    
    if voting_bias is not None:
        scores = [s * sp for s, sp in zip(scores,sys_proportions)]
    # scores = [sp * sp for s, sp in zip(scores,sys_proportions)]

    sorted_hyps = sorted([(s, n, hyp) for hyp, n, s in zip(hyps, sys_names, scores)], reverse=True)
    if verbose > 0:
        print(f"\nOriginal: {source}")
        for i, (s, name, hyp) in enumerate(sorted_hyps):
            print(f"Top {i+ 1} hyp is {name} with {s} score: {hyp}")
        print('\n')
    top_hyp, top_sys = sorted_hyps[0][-1], sorted_hyps[0][-2]  
    return top_hyp, top_sys


def get_choosen_rejected(source, hyps, model, voting_bias=True, m7_bias=0, verbose=0):
    if len(hyps) == 1:
        return hyps[0], hyps[0]
    
    # score hypothesis
    scores = model.score(source, hyps)
    scores = scores.tolist()

    pi = max(enumerate(scores), key=lambda x:x[1])[0]
    ni = min(enumerate(scores), key=lambda x:x[1])[0]

    return hyps[pi], hyps[ni]


In [ ]:
device = "cuda"
path_to_model = "/mnt/green-efs/kostiantyn.omelianchuk/gec_sota/models/greco/models/checkpoint.bin"
model = GRECO('microsoft/deberta-v3-large').to(device)
# model.load_state_dict(torch.load(path_to_model))
weights = torch.load(path_to_model)
model.load_state_dict({n:weights[n] for n, p in model.named_parameters() })

In [ ]:
source = "He go at school ."
hyps = ["He goes to school .", "He goes at school .", "He go at school .", "He go .", ]
# sys_names = ['best', 'second|third', 'orig', 'broken']
# get_top_candidate(source, hyps, sys_names, model, verbose=1)

get_choosen_rejected(source, hyps, model)

In [ ]:
df = pd.read_json('troy-llama7-gec-20k-sample.json')

In [ ]:
src = []
positives = []
negatives = []
samples = []

for _, row in tqdm(df.iterrows()):
    # s = detokenize(row.src.split(" "))
    s = row.src

    p, n = get_choosen_rejected(s, row['pred-llama2-sample'], model)
    
    positives.append(p)
    negatives.append(n)
    src.append(s)

    d = {
        "question": row.src, 
        "answer": [
            p.strip(),
            n.strip()
        ], 
        "system": "Rewrite this text to make it grammatically correct ."
    } 
    samples.append(d)


In [ ]:
import json
# pd.DataFrame(samples).to_json('llama7_gec_greco.json')
with open('llama7_gec_greco.json', "w") as f:
    json.dump(samples, f, indent=2)


In [ ]:
# Finetune LLama with DPO

In [ ]:
git clone https://github.com/hiyouga/LLaMA-Factory.git

In [ ]:
WANDB_DISABLED=true CUDA_VISIBLE_DEVICES=0,1,2,3 accelerate launch \
    --config_file ../accelerate/single_config.yaml \
    ../../src/train_bash.py \
    --stage dpo \
    --do_train \
    --model_name_or_path /home/oleksandr.korniienko/gec-llama2-7b-public/ \
    --create_new_adapter \
    --dataset llama7_gec_greco \
    --dataset_dir ../../data \
    --template llama2 \
    --finetuning_type lora \
    --lora_target q_proj,k_proj,v_proj,o_proj,gate_proj,up_proj \
    --output_dir ../../saves/LLaMA2-7B-gec-greco/dpo-02/ \
    --overwrite_cache \
    --overwrite_output_dir \
    --cutoff_len 512 \
    --preprocessing_num_workers 16 \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --logging_steps 50 \
    --warmup_steps 20 \
    --save_steps 200 \
    --eval_steps 200 \
    --evaluation_strategy steps \
    --load_best_model_at_end \
    --learning_rate 1e-5 \
    --num_train_epochs 2.0 \
    --val_size 0.1 \
    --dpo_ftx 1.0 \
    --plot_loss \
    --fp16 \
    --ddp_timeout 180000000 \
    --ddp_find_unused_parameters False \
    --dpo_beta 0.2


WANDB_DISABLED=true CUDA_VISIBLE_DEVICES=0,1,2,3 accelerate launch \
    --config_file ../accelerate/single_config.yaml \
    ../../src/train_bash.py \
    --stage sft \
    --do_predict \
    --model_name_or_path /home/oleksandr.korniienko/gec-llama2-7b-public/ \
    --adapter_name_or_path ../../saves/LLaMA2-7B-gec-greco/dpo-02/checkpoint-400/ \
    --dataset gec_nucle_test \
    --dataset_dir ../../data \
    --template llama2 \
    --finetuning_type lora \
    --lora_target q_proj,k_proj,v_proj,o_proj,gate_proj,up_proj \
    --output_dir ../../saves/LLaMA2-7B-gec-greco/dpo-02/predict-nucle/ \
    --overwrite_cache \
    --overwrite_output_dir \
    --cutoff_len 256 \
    --preprocessing_num_workers 16 \
    --per_device_eval_batch_size 1 \
    --max_samples 1311 \
    --fp16 \
    --predict_with_generate
    

WANDB_DISABLED=true CUDA_VISIBLE_DEVICES=0,1,2,3 accelerate launch \
    --config_file ../accelerate/single_config.yaml \
    ../../src/train_bash.py \
    --stage sft \
    --do_predict \
    --model_name_or_path /home/oleksandr.korniienko/gec-llama2-7b-public/ \
    --adapter_name_or_path ../../saves/LLaMA2-7B-gec-greco/dpo-02/checkpoint-400/ \
    --dataset gec_bea_dev \
    --dataset_dir ../../data \
    --template llama2 \
    --finetuning_type lora \
    --lora_target q_proj,k_proj,v_proj,o_proj,gate_proj,up_proj \
    --output_dir ../../saves/LLaMA2-7B-gec-greco/dpo-02/predict-bea/ \
    --overwrite_cache \
    --overwrite_output_dir \
    --cutoff_len 256 \
    --preprocessing_num_workers 16 \
    --per_device_eval_batch_size 1 \
    --max_samples 4384 \
    --predict_with_generate

In [ ]:
# Evaluate

In [ ]:
import os
import pandas as pd
import json

import spacy
NLP = spacy.load('en_core_web_sm')

import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

def tokenize(txt):
    doc = NLP(txt)
    return " ".join([t.text for t in doc])

import subprocess
def run(cmd):
    print("Run shell command:\n\t", cmd)
    return subprocess.run(cmd, shell=True)

In [ ]:
# BEA
bea_cmd = """
docker run --rm  -v /mnt/green-efs/kostiantyn.omelianchuk/gec_sota/data/evalsets/bea-dev.txt:/data/input.txt \
    -v /mnt/green-efs/kostiantyn.omelianchuk/gec_sota/data/evalsets/bea-dev.m2:/data/ref.m2 \
    -v "{pred_txt}":/data/pred.txt \
    errant
"""


In [ ]:
# experiment_path = '/home/oleksandr.korniienko/LLaMA-Factory/saves/LLaMA2-7B-gec-greco/dpo-02'
# experiment_path = '/home/oleksandr.korniienko/LLaMA-Factory/saves/LLaMA2-7B-gec-full'


experiments = [
    '/home/oleksandr.korniienko/LLaMA-Factory/saves/LLaMA2-7B-gec-greco/dpo-02',
    '/home/oleksandr.korniienko/LLaMA-Factory/saves/LLaMA2-7B-gec-greco/dpo-05',
    '/home/oleksandr.korniienko/LLaMA-Factory/saves/LLaMA2-7B-gec-greco/dpo-07',
    '/home/oleksandr.korniienko/LLaMA-Factory/saves/LLaMA2-7B-gec-greco/dpo-1',
    '/home/oleksandr.korniienko/LLaMA-Factory/saves/LLaMA2-7B-gec-greco/dpo-4'
]


for experiment_path in experiments:
    print("==================")
    print(experiment_path)

    output_path = os.path.join(experiment_path, 'predict-nucle', 'predictions.csv')

    with open(os.path.join(experiment_path, 'predict-nucle', 'generated_predictions.jsonl'), 'r') as f:
        df = pd.DataFrame( [json.loads(l) for l in f.readlines()] )
    df['output'] = df['predict']
    df['output'] = df.output.map(tokenize)
    df.to_csv(output_path, index=False)
        
    cmd_evaluate = f's2s_evaluate --eval /mnt/green-efs/kostiantyn.omelianchuk/data/evalsets/nucle14-2a.m2 --pred {output_path} --out {experiment_path}/metrics-nucle.yaml  --metrics max_match wed --print-format yaml'
    
    run(cmd_evaluate)

    ######
    output_path = os.path.join(experiment_path, 'predict-bea', 'predictions.txt')
    
    with open(os.path.join(experiment_path, 'predict-bea', 'generated_predictions.jsonl'), 'r') as f:
        df = pd.DataFrame( [json.loads(l) for l in f.readlines()] )
    df['output'] = df['predict']
    df['output'] = df.output.map(tokenize)
    
    with open(output_path, 'w') as f:
        f.write("\n".join(df.output.tolist()))
    run(bea_cmd.format(pred_txt=output_path))

